## SQL Basics
This notebook is a short introduction to SQL. It is based on the [DuckDB](https://duckdb.org/) database engine.

## Datasets
The following datasets are used in this notebook. You don’t need to download them, they can be accessed directly from the notebook.

* [cities.csv](https://open.gishub.org/data/duckdb/cities.csv)
* [countries.csv](https://open.gishub.org/data/duckdb/countries.csv)

## Reference
* [Tutorial from Dr. QiuSheng Wu](https://geog-414.gishub.org/book/duckdb/02_sql_basics.html)
* [W3Schools SQL Tutorial](https://www.w3schools.com/sql)
* [DuckDB SQL Introduction](https://duckdb.org/docs/sql/introduction.html)

## Library Import and Configuration

In [1]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

Set configurations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.

```
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
```

## Connecting to DuckDB

Connect jupysql to DuckDB using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, or a file backed db.    
If your SQL query is one line only, you may use the `%sql` magic command. For multi-line SQL query, you may use the `%%sql` magic command.

In [3]:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

Connecting to 'duckdb:///:memory:'

## Install extensions
Check available DuckDB extensions.

In [4]:
%%sql

SELECT * FROM duckdb_extensions();

Running query in 'duckdb:///:memory:'

extension_name,loaded,installed,install_path,description,aliases
arrow,False,False,,A zero-copy data integration between Apache Arrow and DuckDB,[]
autocomplete,False,False,,Adds support for autocomplete in the shell,[]
aws,False,False,,Provides features that depend on the AWS SDK,[]
azure,False,False,,Adds a filesystem abstraction for Azure blob storage to DuckDB,[]
excel,False,False,,Adds support for Excel-like format strings,[]
fts,False,True,(BUILT-IN),Adds support for Full-Text Search Indexes,[]
httpfs,False,False,,Adds support for reading and writing files over a HTTP(S) connection,"['http', 'https', 's3']"
iceberg,False,False,,Adds support for Apache Iceberg,[]
icu,True,True,(BUILT-IN),Adds support for time zones and collations using the ICU library,[]
inet,False,False,,Adds support for IP-related data types and functions,[]


DuckDB’s [httpfs extension](https://duckdb.org/docs/extensions/httpfs) allows parquet and csv files to be queried remotely over http. This is useful for querying large datasets without having to download them locally. Let’s install the extension and load the extension.

In [5]:
%%sql

INSTALL httpfs;
LOAD httpfs;

Running query in 'duckdb:///:memory:'

Success


## Read online CSV
Use the `httpfs` extension to read the `cities.csv` file from the web.

In [6]:
%%sql

SELECT * FROM 'https://open.gishub.org/data/duckdb/cities.csv';

Running query in 'duckdb:///:memory:'

id,name,country,latitude,longitude,population
1,Bombo,UGA,0.5833,32.5333,75000
2,Fort Portal,UGA,0.671,30.275,42670
3,Potenza,ITA,40.642,15.799,69060
4,Campobasso,ITA,41.563,14.656,50762
5,Aosta,ITA,45.737,7.315,34062
6,Mariehamn,ALD,60.097,19.949,10682
7,Ramallah,PSE,31.90294,35.20621,24599
8,Vatican City,VAT,41.90001,12.44781,832
9,Poitier,FRA,46.58329,0.33328,85960
10,Clermont-Ferrand,FRA,45.77998,3.08001,233050


In [7]:
%%sql

SELECT * FROM 'https://open.gishub.org/data/duckdb/countries.csv';

Running query in 'duckdb:///:memory:'

id,Country,Alpha2_code,Alpha3_code,Numeric_code,Latitude,Longitude
1,Afghanistan,AF,AFG,4,33.0,65.0
2,Albania,AL,ALB,8,41.0,20.0
3,Algeria,DZ,DZA,12,28.0,3.0
4,American Samoa,AS,ASM,16,-14.3333,-170.0
5,Andorra,AD,AND,20,42.5,1.6
6,Angola,AO,AGO,24,-12.5,18.5
7,Anguilla,AI,AIA,660,18.25,-63.1667
8,Antarctica,AQ,ATA,10,-90.0,0.0
9,Antigua and Barbuda,AG,ATG,28,17.05,-61.8
10,Argentina,AR,ARG,32,-34.0,-64.0


## Create Table
Create a table named `cities` from the `cities.csv` file.

In [8]:
%%sql 

CREATE TABLE cities AS SELECT * FROM 'https://open.gishub.org/data/duckdb/cities.csv';

Running query in 'duckdb:///:memory:'

Count


Create a table named `countries` from the `countries.csv` file.

In [9]:
%%sql 

CREATE TABLE countries AS SELECT * FROM 'https://open.gishub.org/data/duckdb/countries.csv';

Running query in 'duckdb:///:memory:'

Count


Display the table content in the database.

In [10]:
%%sql 

FROM cities;

Running query in 'duckdb:///:memory:'

id,name,country,latitude,longitude,population
1,Bombo,UGA,0.5833,32.5333,75000
2,Fort Portal,UGA,0.671,30.275,42670
3,Potenza,ITA,40.642,15.799,69060
4,Campobasso,ITA,41.563,14.656,50762
5,Aosta,ITA,45.737,7.315,34062
6,Mariehamn,ALD,60.097,19.949,10682
7,Ramallah,PSE,31.90294,35.20621,24599
8,Vatican City,VAT,41.90001,12.44781,832
9,Poitier,FRA,46.58329,0.33328,85960
10,Clermont-Ferrand,FRA,45.77998,3.08001,233050


In [11]:
%%sql 

FROM countries;

Running query in 'duckdb:///:memory:'

id,Country,Alpha2_code,Alpha3_code,Numeric_code,Latitude,Longitude
1,Afghanistan,AF,AFG,4,33.0,65.0
2,Albania,AL,ALB,8,41.0,20.0
3,Algeria,DZ,DZA,12,28.0,3.0
4,American Samoa,AS,ASM,16,-14.3333,-170.0
5,Andorra,AD,AND,20,42.5,1.6
6,Angola,AO,AGO,24,-12.5,18.5
7,Anguilla,AI,AIA,660,18.25,-63.1667
8,Antarctica,AQ,ATA,10,-90.0,0.0
9,Antigua and Barbuda,AG,ATG,28,17.05,-61.8
10,Argentina,AR,ARG,32,-34.0,-64.0


## The SQL SELECT statement
The `SELECT` statement is used to select data from a database. Use either `SELECT *` to select all columns, or `SELECT column1, column2, ...` to select specific columns.

`SELECT * FROM cities` is the same as `FROM cities`.

In [12]:
%%sql 

SELECT * FROM cities;

Running query in 'duckdb:///:memory:'

id,name,country,latitude,longitude,population
1,Bombo,UGA,0.5833,32.5333,75000
2,Fort Portal,UGA,0.671,30.275,42670
3,Potenza,ITA,40.642,15.799,69060
4,Campobasso,ITA,41.563,14.656,50762
5,Aosta,ITA,45.737,7.315,34062
6,Mariehamn,ALD,60.097,19.949,10682
7,Ramallah,PSE,31.90294,35.20621,24599
8,Vatican City,VAT,41.90001,12.44781,832
9,Poitier,FRA,46.58329,0.33328,85960
10,Clermont-Ferrand,FRA,45.77998,3.08001,233050


To limit the number of rows returned, use the `LIMIT` keyword. For example, `SELECT * FROM cities LIMIT 10` will return only the first 10 rows.

In [13]:
%%sql

SELECT * FROM cities LIMIT 10;

Running query in 'duckdb:///:memory:'

id,name,country,latitude,longitude,population
1,Bombo,UGA,0.5833,32.5333,75000
2,Fort Portal,UGA,0.671,30.275,42670
3,Potenza,ITA,40.642,15.799,69060
4,Campobasso,ITA,41.563,14.656,50762
5,Aosta,ITA,45.737,7.315,34062
6,Mariehamn,ALD,60.097,19.949,10682
7,Ramallah,PSE,31.90294,35.20621,24599
8,Vatican City,VAT,41.90001,12.44781,832
9,Poitier,FRA,46.58329,0.33328,85960
10,Clermont-Ferrand,FRA,45.77998,3.08001,233050


Select a subset of columns from the `cities` table and display the first 10 rows.

In [14]:
%%sql

SELECT name, country FROM cities LIMIT 10;

Running query in 'duckdb:///:memory:'

name,country
Bombo,UGA
Fort Portal,UGA
Potenza,ITA
Campobasso,ITA
Aosta,ITA
Mariehamn,ALD
Ramallah,PSE
Vatican City,VAT
Poitier,FRA
Clermont-Ferrand,FRA


To select distinct values, use the `DISTINCT` keyword. For example, `SELECT DISTINCT country FROM cities` will return only the distinct values of the `country` column.

In [15]:
%%sql

SELECT DISTINCT country FROM cities LIMIT 10;

Running query in 'duckdb:///:memory:'

country
VAT
NLD
GEO
PRT
PAK
UZB
NER
POL
CHE
EGY


To count the number of rows returned, use the `COUNT(*)` function. For example, `SELECT COUNT(*) FROM cities` will return the number of rows in the `cities` table.

In [16]:
%%sql

SELECT COUNT(*) FROM cities;

Running query in 'duckdb:///:memory:'

count_star()
1249


To count the number of distinct values, use the `COUNT(DISTINCT column)` function. For example, `SELECT COUNT(DISTINCT country) FROM cities` will return the number of distinct values in the `country` column.

In [17]:
%%sql

SELECT COUNT(DISTINCT country) FROM cities;

Running query in 'duckdb:///:memory:'

count(DISTINCT country)
200


To calculate the maximum value, use the `MAX(column)` function. For example, `SELECT MAX(population) FROM cities` will return the maximum value in the `population` column.

In [18]:
%%sql

SELECT MAX(population) FROM cities;

Running query in 'duckdb:///:memory:'

max(population)
35676000


To calculate the total value, use the `SUM(column)` function. For example, `SELECT SUM(population) FROM cities` will return the total value in the `population` column.

In [19]:
%%sql

SELECT SUM(population) FROM cities;

Running query in 'duckdb:///:memory:'

sum(population)
1475534501


To calculate the average value, use the `AVG(column)` function. For example, `SELECT AVG(population) FROM cities` will return the average value in the `population` column.

In [20]:
%%sql

SELECT AVG(population) FROM cities;

Running query in 'duckdb:///:memory:'

avg(population)
1181372.6989591673


To order the results, use the O`RDER BY column` clause. For example, `SELECT * FROM cities ORDER BY country` will return the rows ordered by the `country` column alphabetically.

In [21]:
%%sql

SELECT * FROM cities ORDER BY country LIMIT 10;

Running query in 'duckdb:///:memory:'

id,name,country,latitude,longitude,population
666,Mazar-e Sharif,AFG,36.69999,67.10003,458151
1219,Kabul,AFG,34.51669,69.18326,3277000
665,Herat,AFG,34.33001,62.16999,481009
1024,Kandahar,AFG,31.61002,65.69495,715542
1177,Luanda,AGO,-8.83829,13.23443,5172900
669,Malanje,AGO,-9.54,16.34003,125856
670,Benguela,AGO,-12.57826,13.40723,151226
671,Lubango,AGO,-14.91001,13.49002,125632
672,Namibe,AGO,-15.19004,12.16002,132900
1026,Menongue,AGO,-14.66661,17.69999,13030


To order the results in descending order, use the `ORDER BY column DESC clause`. For example, `SELECT * FROM cities ORDER BY country ASC, population DESC` will return the rows ordered by the `country` column alphabetical order and then by the `population` column in descending order.

In [22]:
%%sql 

SELECT * FROM cities ORDER BY country ASC, population DESC LIMIT 10;

Running query in 'duckdb:///:memory:'

id,name,country,latitude,longitude,population
1219,Kabul,AFG,34.51669,69.18326,3277000
1024,Kandahar,AFG,31.61002,65.69495,715542
665,Herat,AFG,34.33001,62.16999,481009
666,Mazar-e Sharif,AFG,36.69999,67.10003,458151
1177,Luanda,AGO,-8.83829,13.23443,5172900
1027,Huambo,AGO,-12.74999,15.76001,1100000
670,Benguela,AGO,-12.57826,13.40723,151226
672,Namibe,AGO,-15.19004,12.16002,132900
669,Malanje,AGO,-9.54,16.34003,125856
671,Lubango,AGO,-14.91001,13.49002,125632


: 